### コーパスの中身をkmeansでクラスタリング

In [1]:
from collections import defaultdict

from gensim import corpora
from gensim import models
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
model = KeyedVectors.load('model/word2vec_ramen_model.model')

In [3]:
max_vocab = 30000
vocab = list(model.wv.key_to_index.keys())[:max_vocab]
vectors = [model.wv[word] for word in vocab]

In [7]:
# クラスター数はこちらで任意の値を定める
n_clusters = 8
kmeans_model = KMeans(n_clusters=n_clusters, verbose=0, random_state=42)
kmeans_model.fit(vectors)

KMeans(random_state=42)

In [8]:
cluster_labels = kmeans_model.labels_
cluster_to_words = defaultdict(list)
for cluster_id, word in zip(cluster_labels, vocab):
    cluster_to_words[cluster_id].append(word)
    
for label, words in cluster_to_words.items():
    print(label, words[:20])

0 ['スープ', '醤油', '感じ', '濃厚', '煮干し', '旨味', '出汁', '味わい', 'つけ', '香り', '風味', '良く', '魚介', 'なく', 'バランス', '印象', '豚骨', '見た目', 'ストレート', '中太']
1 ['ラーメン', '0円', 'つけ麺', '注文', 'そば', 'メニュー', '中華そば', '特製', 'らーめん', '今回', '購入', '味噌', '提供', '普通', 'ご飯', '限定', '餃子', '生姜', 'なし', 'お願い']
3 ['チャーシュー', '美味しい', '良い', 'メンマ', 'トッピング', 'ネギ', 'ワンタン', '食感', '野菜', '海苔', '具材', '旨い', '美味い', '味付け', '調理', 'タイプ', 'シンプル', '玉子', '相性', 'ニンニク']
4 ['こと', '0分', '0時', 'こちら', '訪問', '到着', '人気', '待ち', '行列', '時間', '今日', 'ごちそうさま', '平日', '開店', 'ため', '近く', '以上', 'ランチ', '満足', '営業']
6 ['さん', '出て', '名前', '大勝軒', '昭和', '町中華', 'あたり', '時代', '展開', 'かき氷', '新しい', '流れ', 'ちゃん', '文字', '勝手', 'ひとつ', '拉麺', '満来', 'たんたん', '居酒屋']
7 ['よう', 'これ', 'そう', 'いい', 'ない', 'もの', 'それ', '好き', 'ところ', 'ここ', '好み', '最後', '美味しかっ', '一杯', 'みたい', '多い', '美味し', '美味しく', '高い', '自分']
5 ['店内', 'カウンター', '食券', '券売機', '店主', '入店', '店員', '雰囲気', '着席', '先客', 'お客さん', '丁寧', '女性', '厨房', '案内', '満席', '綺麗', '接客', 'スタッフ', 'テーブル']
2 ['ラーメン屋', '徒歩', '場所', '食べログ', 'オープン', 'ラーメン店', '名店', '店舗', '東京',

In [11]:
def change_dict_key(d, old_key, new_key, default_value=None):
    d[new_key] = d.pop(old_key, default_value)

In [12]:
# # 各クラスターの解釈を決定
change_dict_key(cluster_to_words, 1,  'ラーメンに関するワード')
change_dict_key(cluster_to_words, 0, 'スープに関するワード')
change_dict_key(cluster_to_words, 3, 'トッピングに関するワード')
change_dict_key(cluster_to_words, 5, '店内に関するワード')
change_dict_key(cluster_to_words, 2, '店舗に関するワード')
change_dict_key(cluster_to_words, 6, '店の特徴に関するワード')
change_dict_key(cluster_to_words, 4, '感想に関するワード')
change_dict_key(cluster_to_words, 7, 'その他のワード')


In [13]:
df = pd.DataFrame.from_dict(cluster_to_words, orient="index").T
df.to_csv('data/words_cluster.csv')
df.head(20)

,ラーメンに関するワード,スープに関するワード,トッピングに関するワード,店内に関するワード,店舗に関するワード,店の特徴に関するワード,感想に関するワード,その他のワード
0,ラーメン,スープ,チャーシュー,店内,ラーメン屋,さん,こと,よう
1,0円,醤油,美味しい,カウンター,徒歩,出て,0分,これ
2,つけ麺,感じ,良い,食券,場所,名前,0時,そう
3,注文,濃厚,メンマ,券売機,食べログ,大勝軒,こちら,いい
4,そば,煮干し,トッピング,店主,オープン,昭和,訪問,ない
5,メニュー,旨味,ネギ,入店,ラーメン店,町中華,到着,もの
6,中華そば,出汁,ワンタン,店員,名店,あたり,人気,それ
7,特製,味わい,食感,雰囲気,店舗,時代,待ち,好き
8,らーめん,つけ,野菜,着席,東京,展開,行列,ところ
9,今回,香り,海苔,先客,有名,かき氷,時間,ここ


### TF-IDFで文章における特徴的な単語を抽出

In [14]:
from gensim import corpora
from gensim import models

In [15]:
# 口コミからレコメンドに使用する情報を絞る
# 試行錯誤して決定するべし!!
ramen_words = cluster_to_words['ラーメンに関するワード']
soup_words = cluster_to_words['スープに関するワード']
topping_words = cluster_to_words['トッピングに関するワード']

ramen_words.extend(soup_words)
ramen_words.extend(topping_words)
key_words = ramen_words

In [16]:
with open('data/ramen_corpus.txt', 'r', encoding='utf-8') as f:
    trainings = []
    for i, data in enumerate(f):
        # 不要な文字を取り除いて単語を取り出す
        words = data.replace("'", '').replace('[', '').replace(']', '').replace(' ', '').replace('\n', '').split(",")
        trainings.append([i for i in words if i in key_words])

In [17]:
# 単語->id変換の辞書作成
dictionary = corpora.Dictionary(trainings)

In [18]:
# textsをcorpus化
corpus = list(map(dictionary.doc2bow, trainings))

In [19]:
# tfidf modelの生成
test_model = models.TfidfModel(corpus)

In [20]:
# corpusへのモデル適用
corpus_tfidf = test_model[corpus]

In [21]:
# id->単語へ変換
texts_tfidf = []    # id -> 単語表示に変えた文書ごとのTF-IDF
for doc in corpus_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([dictionary[word[0]], word[1]])
    texts_tfidf.append(text_tfidf)

In [22]:
from operator import itemgetter
import pickle

In [23]:
texts_tfidf_sorted_top20 = [] 

#TF-IDF値を高い順に並び替え上位単語20個に絞る。
for i in range(len(texts_tfidf)):
    soted = sorted(texts_tfidf[i], key=itemgetter(1),reverse=True)
    soted_top20 = soted[:20]
    word_list = []
    for k in range(len(soted_top20)):
        word = soted_top20[k][0]
        word_list.append(word)
    texts_tfidf_sorted_top20.append(word_list)

In [24]:
# 結果をデータフレームに追加
store_df = pd.read_csv('data/ramen_store.csv')
review_df = pd.read_csv('data/ramen_review.csv')
ramen_df = pd.merge(store_df, review_df, on='ID')

In [25]:
ramen_df = ramen_df.groupby(['ID', 'name', 'url', 'score'])['review'].apply(list).apply(' '.join).reset_index().sort_values('ID')
ramen_df['texts_tfidf_sorted_top20'] = texts_tfidf_sorted_top20
ramen_df.shape

(500, 6)

In [26]:
df_ramen_texts_tfidf_sorted_top20 = ramen_df[['ID', 'texts_tfidf_sorted_top20']]
df_ramen_texts_tfidf_sorted_top20.head()

,ID,texts_tfidf_sorted_top20
0,1,"[ワンタン, 炭火, 昆布, 海老, 特製, 火炙り, だれ, 藻塩, 焼き豚, 炙り焼, ..."
1,2,"[黒毛和牛, いりこ, spf, ワンタン, 鰹節, チャー, 伊吹, めん, 書き, 地鶏..."
2,3,"[ワンタン, 中華そば, 三つ葉, 特製, 鶏胸肉, 生醤油, 紫蘇, 大葉, 塩か, むね..."
3,4,"[軍鶏, 替え玉, そば, まぜそば, 卵黄, 素地, 燻製, 特製, 醤油漬け, レモン,..."
4,5,"[しじみ, 中華蕎麦, シジミ, 琥珀, ワンタン, バラ, コハク酸, 地鶏, 特製, 菅..."


In [27]:
# 上の結果をダンプ
with open('data/df_ramen_texts_tfidf_sorted_top20', 'wb') as f:
    pickle.dump(df_ramen_texts_tfidf_sorted_top20, f)

In [28]:
ramen_df[['ID', 'name', 'url', 'texts_tfidf_sorted_top20']].to_csv('data/tfidf_top20.csv')